In [1]:
from utils import *
import numpy as np
import torch, os
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
set_random_seed(42)

Seed: 42


In [3]:
# def scaling_embeddings(embeddings):
#     for i in range(len(embeddings)):
#         embeddings[i] = StandardScaler().fit_transform(embeddings[i])
#     return embeddings

def reshape_embeddings(embeddings):
    if embeddings.shape[1] == 1:
        embeddings = embeddings.squeeze(1)
    return embeddings

In [4]:
dynahate_labels_train = process_labels(read_labels("dynahate", "train"))
dynahate_labels_dev = process_labels(read_labels("dynahate", "dev"))
dynahate_labels_test = process_labels(read_labels("dynahate", "test"))

latenthatred_labels_train = read_labels("latenthatred", "train")
latenthatred_labels_dev = read_labels("latenthatred", "dev")
latenthatred_labels_test = read_labels("latenthatred", "test")

olid_labels_train = read_labels("olid", "train")
olid_labels_dev = read_labels("olid", "dev")
olid_labels_test = read_labels("olid", "test")

In [5]:
mlp = MLPClassifier(random_state=42)
gridsearch = GridSearchCV(
    mlp,
    param_grid={
        "hidden_layer_sizes": [(128), (128,64)],
        "activation": ["relu"],
        "solver": ["adam"],
        "learning_rate_init": [0.001, 0.0001],
        "learning_rate": ["adaptive"],
        "early_stopping": [True],
        "max_iter": [10000]
    },
    verbose=4,
    n_jobs=os.cpu_count()//3,
    cv=5,
)

In [6]:
dynahate_labels_train_dev = np.concatenate((dynahate_labels_train, dynahate_labels_dev))
latenthatred_labels_train_dev = np.concatenate((latenthatred_labels_train, latenthatred_labels_dev))
olid_labels_train_dev = np.concatenate((olid_labels_train, olid_labels_dev))

In [7]:
hatebert_dynahate_train_embeddings = reshape_embeddings(hatebert_dynahate_train_embeddings)
hatebert_dynahate_dev_embeddings = reshape_embeddings(hatebert_dynahate_dev_embeddings)
hatebert_dynahate_test_embeddings = reshape_embeddings(hatebert_dynahate_test_embeddings)

hatebert_latenthatred_train_embeddings = reshape_embeddings(hatebert_latenthatred_train_embeddings)
hatebert_latenthatred_dev_embeddings = reshape_embeddings(hatebert_latenthatred_dev_embeddings)
hatebert_latenthatred_test_embeddings = reshape_embeddings(hatebert_latenthatred_test_embeddings)

hatebert_olid_train_embeddings = reshape_embeddings(hatebert_olid_train_embeddings)
hatebert_olid_dev_embeddings = reshape_embeddings(hatebert_olid_dev_embeddings)
hatebert_olid_test_embeddings = reshape_embeddings(hatebert_olid_test_embeddings)

### HateBERT

#### DynaHate

In [8]:
hatebert_dynahate_train_dev = np.concatenate((hatebert_dynahate_train_embeddings, hatebert_dynahate_dev_embeddings))

grid_results = gridsearch.fit(hatebert_dynahate_train_dev, dynahate_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(hatebert_dynahate_train_dev)
test_preds = mlp.predict(hatebert_dynahate_test_embeddings)

computeAllScores(train_dev_preds, test_preds, dynahate_labels_train_dev, dynahate_labels_test, "Results/hatebert_dynahate_seed_42")
save_model(mlp, "Saves/hatebert_dynahate_seed_42")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': 128, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 10000, 'solver': 'adam'}
Accuracy Train Dev:  0.7054883318928262
Accuracy Test:  0.6881067961165048
Weighted F1 Train Dev:  0.7056457637472098
Weighted F1 Test:  0.6885700819497024
Macro F1 Train Dev:  0.7041415347275866
Macro F1 Test:  0.685929987338604
Micro F1 Train Dev:  0.7054883318928262
Micro F1 Test:  0.6881067961165048
Weighted Recall Train Dev:  0.7054883318928262
Weighted Recall Test:  0.6881067961165048
Macro Recall Train Dev:  0.7044040791902362
Macro Recall Test:  0.6867017305282244
Micro Recall Train Dev:  0.7054883318928262
Micro Recall Test:  0.6881067961165048
Confusion Matrix Train Dev: 
[[11811  5306]
 [ 5598 14309]]
Confusion Matrix Test: 
[[1246  606]
 [ 679 1589]]


#### LatentHatred

In [9]:
hatebert_latenthatred_train_dev = np.concatenate((hatebert_latenthatred_train_embeddings, hatebert_latenthatred_dev_embeddings))

grid_results = gridsearch.fit(hatebert_latenthatred_train_dev, latenthatred_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(hatebert_latenthatred_train_dev)
test_preds = mlp.predict(hatebert_latenthatred_test_embeddings)

computeAllScores(train_dev_preds, test_preds, latenthatred_labels_train_dev, latenthatred_labels_test, "Results/hatebert_latenthatred_seed_42")
save_model(mlp, "Saves/hatebert_latenthatred_seed_42")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': (128, 64), 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 10000, 'solver': 'adam'}
Accuracy Train Dev:  0.7103041589075109
Accuracy Test:  0.68975791433892
Weighted F1 Train Dev:  0.6768567113678088
Weighted F1 Test:  0.6522328553750266
Macro F1 Train Dev:  0.47036377403012125
Macro F1 Test:  0.4366347891209231
Micro F1 Train Dev:  0.7103041589075109
Micro F1 Test:  0.68975791433892
Weighted Recall Train Dev:  0.7103041589075109
Weighted Recall Test:  0.68975791433892
Macro Recall Train Dev:  0.4633852066723381
Macro Recall Test:  0.43679655131169554
Micro Recall Train Dev:  0.7103041589075109
Micro Recall Test:  0.68975791433892
Confusion Matrix Train Dev: 
[[9092  845    8]
 [3018 2316   13]
 [ 410  373   35]]
Confusion Matrix Test: 
[[3024  318    4]
 [1073  674    6]
 [ 154  111    6]]


#### OLID

In [10]:
hatebert_olid_train_dev = np.concatenate((hatebert_olid_train_embeddings, hatebert_olid_dev_embeddings))

grid_results = gridsearch.fit(hatebert_olid_train_dev, olid_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(hatebert_olid_train_dev)
test_preds = mlp.predict(hatebert_olid_test_embeddings)

computeAllScores(train_dev_preds, test_preds, olid_labels_train_dev, olid_labels_test, "Results/hatebert_olid_seed_42")

save_model(mlp, "Saves/hatebert_olid_seed_42")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': 128, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 10000, 'solver': 'adam'}
Accuracy Train Dev:  0.7627643504531723
Accuracy Test:  0.791860465116279
Weighted F1 Train Dev:  0.7577082807468295
Weighted F1 Test:  0.7756450810442447
Macro F1 Train Dev:  0.722018376937352
Macro F1 Test:  0.7048192655571983
Micro F1 Train Dev:  0.7627643504531721
Micro F1 Test:  0.791860465116279
Weighted Recall Train Dev:  0.7627643504531723
Weighted Recall Test:  0.791860465116279
Macro Recall Train Dev:  0.714754730563554
Macro Recall Test:  0.6858198924731183
Micro Recall Train Dev:  0.7627643504531723
Micro Recall Test:  0.791860465116279
Confusion Matrix Train Dev: 
[[7584 1256]
 [1885 2515]]
Confusion Matrix Test: 
[[574  46]
 [133 107]]
